In [5]:
import os
import sys
# Add parent directory to sys.path
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("/home/azureuser/cloudfiles/code/Users/shayoni.x.dutta/Test_LLM_FinCONV/agent_backend/azureml/agents/agent_langchain.ipynb"), "..")))
# Now import agent_config
from agent_config import *



27c4fbad0a904765b82b87838210bcec
/mnt/batch/tasks/shared/LS_root/mounts/clusters/shayonisimms/code


In [6]:
cmd = f'pip install -r /home/azureuser/cloudfiles/code/Users/shayoni.x.dutta/Test_LLM_FinCONV/agent_backend/azureml/requirements.txt --quiet'
!{cmd}

In [7]:
%pip install langchain langchain-openai pandasql --quiet
%pip install duckdb --quiet
#%pip install langchain-community --quiet

Note: you may need to restart the kernel to use updated packages.


In [74]:




import json
import os
from typing import Any, List, Optional, Type

import mlflow  # type: ignore
import pandas as pd
# from databricks.sdk.runtime import *  # type: ignore # noqa
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.agents.agent import RunnableAgent, RunnableMultiActionAgent
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from pydantic import BaseModel
from langchain.tools import BaseTool
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)
from langchain_openai import AzureChatOpenAI
# from mlflow.types.llm import (  # Non-streaming helper classes
#     ChatChoice,
#     ChatCompletionResponse,
#     ChatMessage,
#     ChatParams,
# )
#from pandasql import sqldf
import duckdb


from utils import *

Note: you may need to restart the kernel to use updated packages.


In [27]:
# Running the agent_config notebook to iniate the constants

In [9]:
# MLFlow Setup

experiment = mlflow.set_experiment("/home/azureuser/cloudfiles/code/Users/shayoni.x.dutta/Test_LLM_FinCONV/agent_backend/azureml")  # type: ignore  # noqa

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
2025/05/20 13:44:15 INFO mlflow.tracking.fluent: Experiment with name '/home/azureuser/cloudfiles/code/Users/shayoni.x.dutta/Test_LLM_FinCONV/agent_backend/azureml' does not exist. Creating a new experiment.


In [10]:
token_provider = get_bearer_token_provider_kong()

In [11]:
files_metadata, csv_files_data = generate_source_data(f'/home/azureuser/cloudfiles/code/Users/shayoni.x.dutta/Test_LLM_FinCONV/agent_backend/datasources')

In [8]:
# Setting up the agent's tools

In [59]:
from pydantic import Field

class GetDatasetMetadaInput(BaseModel):
    """
    Abstract base class for creating input for agent tools.

    Attributes:
        <var> (<type>): Variable names that tool will use as input arguments.

    """

    question: Optional[str] = Field(description="The prompt/request from the user", default=None)


class GetDatasetMetadaTool(BaseTool):
    """
    Abstract base class for creating agent tools.

    Attributes:
        name (str): The name of the agent tool.
        description (str): A brief description of the agent tool.
        args_schema (Type[BaseModel]): The schema model for the arguments the agent tool accepts.


    """

    name: str = "get_file_metadata"
    description: str = (
        "Provides the metada, (like description, schema and sample) of the dataset (multiple CSV files) to be used for inference by the agent."
    )
    args_schema: Type[BaseModel] = GetDatasetMetadaInput

    def _run(self, question: Optional[str] = None) -> Any:
        """
        Synchronously run the agent tool to provide the appropriate file name and file schema to
        answer the user's question.

        Args:
            question (str): The question asked by user in input terminal.

        Returns:
            It provides a dictionary with the metadata (file name, description, schema and sample).
        """

        return json.dumps(files_metadata)

In [68]:
class GetTargetFileSchemaInput(BaseModel):
    """
    Abstract base class for creating input for agent tools.

    Attributes:
        <var> (<type>): Variable names that tool will use as input arguments.

    """

    question: str = Field(description="The prompt/request from the user")
    files_metadata: str = Field(
        description="A dictionary containing the metadata about the CSV files in the dataset."
    )


class GetTargetFileSchemaTool(BaseTool):
    """
    Abstract base class for creating agent tools.

    Attributes:
        name (str): The name of the agent tool.
        description (str): A brief description of the agent tool.
        args_schema (Type[BaseModel]): The schema model for the arguments the agent tool accepts.


    """

    name: str = "get_target_file_schema"
    description: str = (
        "Provides the most likely file_name and file_schema to be used for querying based on \
         the input question and the files/dataset metadata."
    )
    args_schema: Type[BaseModel] = GetTargetFileSchemaInput

    def _run(self, question: str, files_metadata: str) -> Any:
        """
        Synchronously run the agent tool to provide the appropriate file name and file schema to
        answer the user's question.

        Args:
            question (str): The question asked by user in input terminal.

        Returns:
            It provides with relevant file_name and file schema from where question belongs to.
        """

        response_schemas = [
            ResponseSchema(
                name="file_name",
                description="The selected file's name.",
            ),
            ResponseSchema(
                name="file_schema",
                description="The select file's schema.",
            ),
        ]

        output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
        format_instructions = output_parser.get_format_instructions()

        format_instructions += (
            ' make sure to format it properly to be used as JSON, like enclosing strings with ".'
        )

        prompt_text = """
        Given the files listed below and their respective file schema, select the most \
        relevant file to answer the following question: '{question}'.
        {files_metadata}.
        {format_instructions}.
        You can remove the sample data from the file_schema, just pass a string with each column separated by commas.
        """

        prompt = PromptTemplate.from_template(
            template=prompt_text,
            partial_variables={
                "format_instructions": format_instructions,
                "files_metadata": files_metadata,
            },
        )
        _input = {"question": question}

        model = AzureChatOpenAI(
            model="o1",
            azure_endpoint="https://dev.api.gsk.com/co/psc/azureopenai/us6/", # os.getenv("o1_AZURE_OPENAI_ENDPOINT"),
            azure_deployment="o1", # os.getenv("o1_AZURE_OPENAI_DEPLOYMENT"),
            api_version= "2024-12-01-preview", # os.getenv("o1_AZURE_OPENAI_API_VERSION"),
            azure_ad_token_provider=token_provider
        )

        chain = prompt | model | output_parser

        res = chain.invoke(_input)

        return res

In [69]:
class GenerateSQLForFileInput(BaseModel):
    """
    Abstract base class for creating input for agent tools.

    Attributes:
        <var> (<type>): Variable names that tool will use as input arguments.

    """

    question: str = Field(
        description="The prompt/request from the user to generate the SQL query for."
    )
    file_name: str = Field(
        description="The file name that represents the table name to be used in the SQL query"
    )
    file_schema: str = Field(
        description="A dictionary represented as string representing a sample of the file's \
                     schema to be used in the SQL query"
    )


class GenerateSQLForFileTool(BaseTool):
    """
    Abstract base class for creating agent tools.

    Attributes:
        name (str): The name of the agent tool.
        description (str): A brief description of the agent tool.
        args_schema (Type[BaseModel]): The schema model for the arguments the agent tool accepts.


    """

    name: str = "generate_sql_for_file"
    description: str = (
        """Provides the SQL query to used to answer the user's request based on a \
        file_name (that represent the table name) and a file_schema \
        (that represents the colums in the table)"""
    )
    args_schema: Type[BaseModel] = GenerateSQLForFileInput

    def _run(self, question: str, file_name: str, file_schema: str) -> Any:
        """
        Synchronously run the agent tool to provide sql query to be ran on the datasource.

        Args:
            question (str): The question asked by user in input terminal.
            file_name (str): Relevant file name generated by GetTargetFileSchemaTool.
            file_schema (str): Relevant file schema generated by GetTargetFileSchemaTool.

        Returns:
            It provides with relevant SQL query with file_name as table and column name
            refers in file_schema.
        """

        response_schemas = [
            ResponseSchema(
                name="sql_query",
                description="SQL query that will be returned",
            ),
        ]

        output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
        format_instructions = output_parser.get_format_instructions()

        format_instructions += " make sure to format it properly to be used as plain string \
                                 like one sentence with SQL query."

        prompt_text = """
        Given the following file name '{file_name}', that is supposed
        to be used as the SQL table name
        and the following JSON representing the schema of the columns in that
        table:

        {file_schema}

        Please provide a SQL query that answers the following question: '{question}'.

        When string values are to be used as filters, make sure
        both the column and value have lower case to avoid case sensitive issues.

        Just answer with the SQL query and nothing else.

        {format_instructions}

        """

        model = AzureChatOpenAI(
            model="o1",
            azure_endpoint="https://dev.api.gsk.com/co/psc/azureopenai/us6/", # os.getenv("o1_AZURE_OPENAI_ENDPOINT"),
            azure_deployment="o1", # os.getenv("o1_AZURE_OPENAI_DEPLOYMENT"),
            api_version= "2024-12-01-preview", # os.getenv("o1_AZURE_OPENAI_API_VERSION"),
            azure_ad_token_provider=token_provider
        )

        prompt = PromptTemplate.from_template(
            template=prompt_text, partial_variables={"format_instructions": format_instructions}
        )
        _input = {"question": question, "file_name": file_name, "file_schema": file_schema}

        chain = prompt | model | output_parser

        res = chain.invoke(_input)["sql_query"]

        return res

In [75]:
class GenerateAnswerforQuestionInput(BaseModel):
    """

    Abstract base class for creating input for agent tools.

    Attributes:
        <var> (<type>): Variable names that tool will use as input arguments.

    """

    question: str = Field(description="The initial prompt/request from the user.")
    sql_query: str = Field(description="The SQL query to be executed on the database")
    file_name: str = Field(
        description="The file name that represents the table name to be used in the SQL query"
    )
    file_schema: str = Field(
        description="A dictionary represented as string representing a sample of the file's \
                     schema to be used in the SQL query"
    )


class GenerateAnswerforQuestionTool(BaseTool):
    """
    Abstract base class for creating agent tools.

    Attributes:
        name (str): The name of the agent tool.
        description (str): A brief description of the agent tool.
        args_schema (Type[BaseModel]): The schema model for the arguments the agent tool accepts.

    """

    name: str = "generate_answer_for_question"
    description: str = """Provides answer once the SQL query gets executed over database"""
    args_schema: Type[BaseModel] = GenerateAnswerforQuestionInput

    def _run(self, question: str, sql_query: str, file_name: str, file_schema: str) -> Any:
        """
        Synchronously run the agent tool to provide the result from sql query..

        Args:
            question (str): The question asked by user in input terminal.
            file_name (str): Relevant file name generated by GetTargetFileSchemaTool.
            file_schema (str): Relevant file schema generated by GetTargetFileSchemaTool.
            sql_query (str): Relevant sql query generated from  GenerateSQLForFileTool

        Returns:
            It execute sql query and provides result in dict format.
        """

        parse_sql_query = sql_query.split(" ")

        parse_sql_query[parse_sql_query.index("FROM") + 1] = "dftbl"

        new_query = " ".join(parse_sql_query)

        dftbl = pd.DataFrame(csv_files_data[file_name])

        #result_df = sqldf(new_query, locals())
        result_df = duckdb.query(new_query).to_df()


        return result_df.to_dict()

In [77]:
from typing import Type, Any
from pydantic import BaseModel, Field
import pandas as pd
import duckdb


class GenerateAnswerforQuestionInput(BaseModel):
    """
    Input schema for GenerateAnswerforQuestionTool.

    Attributes:
        question (str): The initial prompt/request from the user.
        sql_query (str): The SQL query to be executed on the database.
        file_name (str): The file name that represents the table name to be used in the SQL query.
        file_schema (str): A dictionary represented as a string showing a sample schema of the file.
    """
    question: str = Field(description="The initial prompt/request from the user.")
    sql_query: str = Field(description="The SQL query to be executed on the database.")
    file_name: str = Field(
        description="The file name that represents the table name to be used in the SQL query."
    )
    file_schema: str = Field(
        description="A dictionary represented as string representing a sample of the file's schema to be used in the SQL query."
    )


class GenerateAnswerforQuestionTool(BaseTool):
    """
    Tool to execute SQL query on a dataframe extracted from a file and return the result.

    Attributes:
        name (str): Name of the tool.
        description (str): Description of what the tool does.
        args_schema (Type[BaseModel]): Pydantic schema for expected inputs.
    """

    name: str = "generate_answer_for_question"
    description: str = "Provides answer once the SQL query gets executed over database"
    args_schema: Type[BaseModel] = GenerateAnswerforQuestionInput

    def _run(self, question: str, sql_query: str, file_name: str, file_schema: str) -> Any:
        """
        Executes a SQL query on the loaded dataframe and returns the result.

        Args:
            question (str): User's question.
            sql_query (str): SQL query to be executed.
            file_name (str): CSV file name to load as a dataframe.
            file_schema (str): Schema of the file (not used in query execution).

        Returns:
            dict: The result of the query as a dictionary.
        """
        # Split query and search for 'FROM' (case-insensitive)
        tokens = sql_query.split(" ")
        upper_tokens = [token.upper() for token in tokens]

        try:
            from_index = upper_tokens.index("FROM")
            tokens[from_index + 1] = "dftbl"
        except ValueError:
            raise ValueError("The SQL query does not contain a FROM clause.")

        new_query = " ".join(tokens)

        # Load CSV file into dataframe
        dftbl = pd.DataFrame(csv_files_data[file_name])

        # Run query using duckdb
        result_df = duckdb.query(new_query).to_df()

        return result_df.to_dict()


In [46]:
# Setting up the agent's function

In [71]:
from mlflow.types.llm import ChatMessage, ChatParams, ChatChoice

def my_agent_function(
    messages: List[ChatMessage], params: Optional[ChatParams] = None, verbose: bool = False
) -> dict:
    """
    Executes a request using a chat agent with designated tools to generate responses.

    Args:
        messages (List[ChatMessage]): A list of ChatMessage objects representing the
                                       conversation history.
        params (Optional[ChatParams]): Optional parameters for the chat. Defaults to None.
        verbose (bool): If True, enables verbose logging. Defaults to False.

    Returns:
        ChatCompletionResponse: The generated response from the chat agent.

    Raises:
        ValueError: If the messages list is empty.
    """
    if not messages:
        raise ValueError("The messages list cannot be empty.")

    ask = messages[-1].content  # MFlow Signature requires both inputs to be arrays

    chat_history = [
        {"role": message.role, "content": message.content} for message in messages[:-1]
    ]

    agent_tools = [
        GetDatasetMetadaTool(),  # noqa
        GetTargetFileSchemaTool(),  # noqa
        GenerateSQLForFileTool(),  # noqa
        GenerateAnswerforQuestionTool(),  # noqa
    ]

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a helpful agent who receives questions and answers them
                    with the tools available. If none of your tools are able to provide
                    the necessary answer, respond saying you are not able to assist with
                    that question based on the tools available to you.""",
            ),
            MessagesPlaceholder("chat_history", optional=True),
            ("human", "{input}"),
            MessagesPlaceholder("agent_scratchpad"),
        ]
    )
    #ws.environ["KONG_API_GATEWAY_ENDPOINT"] = keyvault.get_secret(name="flow-ai-id")
    model = AzureChatOpenAI(
        model="o1",
        azure_endpoint="https://dev.api.gsk.com/co/psc/azureopenai/us6/", # os.getenv("o1_AZURE_OPENAI_ENDPOINT"),
        azure_deployment="o1", # os.getenv("o1_AZURE_OPENAI_DEPLOYMENT"),
        api_version= "2024-12-01-preview", # os.getenv("o1_AZURE_OPENAI_API_VERSION"),
        azure_ad_token_provider=token_provider
        )
    

    agent = create_openai_tools_agent(model, agent_tools, prompt)

    agent_executor = AgentExecutor(
        agent=agent,
        tools=agent_tools,
        verbose=verbose,
        return_intermediate_steps=True,
        handle_parsing_errors=True,
    )

    if isinstance(agent_executor.agent, RunnableMultiActionAgent) or isinstance(
        agent_executor.agent, RunnableAgent
    ):
        agent_executor.agent.stream_runnable = False

    res = agent_executor.invoke({"input": ask, "chat_history": chat_history})["output"]

    return {
        "choices": [
            {
                "message": {
                    "role": "assistant",
                    "content": res
                }
            }
        ]
    }

In [64]:
class CustomTokenProvider:
    def __init__(self, access_token):
        self.access_token = access_token
 
    def __call__(self, *scopes):
        return self.access_token
 
 
def create_bearer_token():
 
    federation_url = "https://federation-qa.gsk.com/as/token.oauth2" # os.getenv("federation_url_token")
    client_id = os.environ.get("azure_openai_client") # os.getenv("AZURE_CLIENT_ID")
    client_secret = os.environ.get("azure_openai_secret")
    grant_type = "client_credentials" # os.getenv("grant_type_token")
    client_id = keyvault.get_secret(name="flow-ai-id")
    client_secret= keyvault.get_secret(name="flow-ai-access")
    # Data to send in the request
    token_data = {
        "grant_type": grant_type,
        "client_id": client_id,
        "client_secret": client_secret
    }
 
    # Request a bearer token
    token_response = requests.post(federation_url, data=token_data)
 
    # Extract the access token from the response
    if token_response.status_code == 200:
        token_info = token_response.json()
        access_token = token_info["access_token"]
        #print(f"Access Token: {access_token}")
        return access_token
    else:
        #print(f"Failed to get token: {token_response.text}")
        return "Requst Failed"
 
# Your existing access token
access_token = create_bearer_token()
 
# Create the custom token provider
token_provider = CustomTokenProvider(access_token)
 
def model_client():
 
    gpt4o_model_client = AzureChatOpenAI(
    model="o1",
    azure_endpoint="https://dev.api.gsk.com/co/psc/azureopenai/us6/", # os.getenv("o1_AZURE_OPENAI_ENDPOINT"),
    azure_deployment="o1", # os.getenv("o1_AZURE_OPENAI_DEPLOYMENT"),
    api_version= "2024-12-01-preview", # os.getenv("o1_AZURE_OPENAI_API_VERSION"),
    model_capabilities={"function_calling": True, "vision": True, "json_output": True},
    azure_ad_token_provider=token_provider
    )
 
    return gpt4o_model_client
    
def create_openai_client():
    # Create OpenAI clientCha
    client = AzureOpenAI(
        api_version= "2024-12-01-preview", # os.getenv("o1_AZURE_OPENAI_API_VERSION"),
        azure_endpoint="https://dev.api.gsk.com/co/psc/azureopenai/us6/", # os.getenv("o1_AZURE_OPENAI_ENDPOINT"),
        azure_ad_token= create_bearer_token(),
        max_retries=4
    )
    return client

In [78]:
# To test the agent uncomment the below
#import os
# os.environ["KONG_CLIENT_ID"] = keyvault.get_secret(name="flow-ai-id")  # <-- Set your actual client ID here
# os.environ["KONG_CLIENT_SECRET"] = keyvault.get_secret(name="flow-ai-access")  # <-- Set your actual client secret here

input_messages = [
    ChatMessage(role="user", content="How many asian patients in the dataset?"),
]
#params_with_custom_inputs = ChatParams(custom_inputs={"client_type": "mobile"})
params_with_custom_inputs = ChatParams()

my_agent_function(messages=input_messages, params=params_with_custom_inputs, verbose=True)



> Entering new AgentExecutor chain...

Invoking: `get_file_metadata` with `{'question': 'How many asian patients in the dataset?'}`


{"demographics": {"description": "Includes case-related demographic details such as ethnicity, gender, race, year of birth, and year of death. This data is essential for analyzing trends and patterns across different demographic groups, assisting in decision-making and resource allocation. The table helps understand the diversity and characteristics of the studied population, providing insights into disparities or specific demographic needs.", "sample_schema": [{"case_id": "0030a28c-81aa-44b0-8be0-b35e1dcbf98c", "ethnicity": "not hispanic or latino", "gender": "female", "race": "asian", "year_of_birth": 1952.0, "year_of_death": 2002.0, "file_id": "0bd755e8-752f-49f0-9574-ea7813a7f0e0"}]}, "diagnoses": {"description": "Includes data on tumor diagnoses, such as tumor classification, primary diagnosis, tissue or organ of origin, treatments administered, t

{'choices': [{'message': {'role': 'assistant',
    'content': 'There are 627 Asian patients in the dataset.'}}]}

In [0]:
# Another test

# input_messages = [
#     ChatMessage(role="user", content="Describe the underlying datasources of the dataset."),
# ]
# params_with_custom_inputs = ChatParams(custom_inputs={"client_type": "mobile"})

# print(my_agent_function(messages=input_messages, params=params_with_custom_inputs).choices[0].message.content)